##### Flow:
```text
User Question
    ↓  
Embed question  
    ↓  
Vector similarity search
    ↓  
Top-K chunks
    ↓  
Prompt assembly
    ↓  
LLM call
    ↓  
Final answer
```

##### Prequisite
You will have two Azure OpenAI deployments:

Create in Azure AI Foundry / Azure OpenAI:
- Embedding deployment (example name): text-embedding-3-small
- Chat deployment (example name): gpt-4o-mini

Azure OpenAI API calls use deployment names (not the underlying model name).

##### Input and Output:
- Input:
    - Table: `databricks_rag_demo.default.azure_compute_doc_embeddings`
    - User question

- Output: User's answer

Important design choice: How to do vector search?

We have 3 options:

- Option A — Naive cosine similarity in Spark (we'll start here)
	- Simple
	- Transparent
	- Works for thousands of chunks
	- Good for learning

- Option B — Databricks Vector Search
	- Production-grade
	- Scalable
	- Index-based
	- We can move here later

- Option C — External vector DB (Pinecone, Weaviate, etc.)
	- Overkill right now

We'll start with Option A, then upgrade.

#### 04 - Retrieval-Augmented Generation (RAG)

This notebook implements a basic RAG pipeline:
1. Embed user queries
2. Retrieve relevant document chunks
3. Assemble a grounded prompt
4. Generate an answer using an LLM

In [0]:
%run ./00_install_deps_and_restart

All required packages already installed. No restart needed.


In [0]:
%run ./00_constants

In [0]:
%run ./00_utils

In [0]:
%run ./00_init_openai_client

In [0]:
import mlflow
# Disable mlflow autologging
mlflow.autolog(disable=True)
mlflow.openai.autolog(disable=True)


Vector Search (production retrieval)

3A) Create the Vector Search endpoint + index (recommended via UI)

Fastest/least pain is UI:
- Databricks → Mosaic AI / Vector Search
- Create endpoint (name example): vs_azure_compute
- Create index
  - Type: Delta Sync index
  - Source table: databricks_rag_demo.default.azure_compute_doc_embeddings
  - Primary key: chunk_id
	- Embedding column: embedding

3B) Query the index in Python

Once endpoint + index exist, use the Vector Search client.


In [0]:
questions = [
    "What is the difference between a normal Azure VM and an ephemeral VM?",
    "How do I resize an Azure virtual machine?",
    "What is Azure VM Scale Sets?",
    "How does Azure handle VM disk persistence?",
    "What is the difference between Spot VM and normal VM?",
    # "How do I enable accelerated networking?",
    # "What is Azure availability set vs availability zone?"
]

batch_ask(questions, k=5, retrievers=["A"], do_eval=True, verbose=False)


Question: What is the difference between a normal Azure VM and an ephemeral VM?

--- Retriever A ---

Question: How do I resize an Azure virtual machine?

--- Retriever A ---

Question: What is Azure VM Scale Sets?

--- Retriever A ---

Question: How does Azure handle VM disk persistence?

--- Retriever A ---

Question: What is the difference between Spot VM and normal VM?

--- Retriever A ---


[{'query_id': 'aa9f09f7-ffca-47b8-a349-98b7cc2cddd8',
  'question': 'What is the difference between a normal Azure VM and an ephemeral VM?',
  'answer': 'The primary difference between a normal Azure VM and an ephemeral VM lies in how the operating system disk is managed and the persistence of data.\n\n1. **Operating System Disk**:\n   - **Normal Azure VM**: A standard Azure VM typically uses a managed OS disk that is persistent. This means that the OS disk retains its data even when the VM is stopped or deallocated. The OS disk can be backed up, and data on it is preserved across reboots and maintenance events.\n   - **Ephemeral VM**: An ephemeral VM, on the other hand, uses an ephemeral OS disk, which is a temporary disk that is not persistent. This means that the data on the OS disk is lost if the VM is stopped or deallocated. Ephemeral disks are designed for scenarios where the OS can be quickly redeployed, and the loss of data is acceptable.\n\n2. **Use Cases**:\n   - **Normal Azu